# packages

In [ ]:

from datetime import datetime
import matplotlib.pyplot as plt
import pandas as pd
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import MetaTrader5 as mt5
import pytz
import json

In [ ]:
z = json.load(open('logins.json', 'r'))

# mt5 experimental portfolio

## connect to MT5

In [ ]:
if not mt5.initialize():
    print("initialize() failed")
    mt5.shutdown()

In [ ]:

# request connection status and parameters
print(mt5.terminal_info())
# get data on MetaTrader 5 version
print(mt5.version())

In [ ]:

# login to my account and print account info
account=z[0]['account_number']
server = z[0]['server']
password = z[0]['password']
authorized=mt5.login(login = account,server = server, password=password)
if authorized:
	# display trading account data in the form of a list
	print("Show account_info()._asdict():")
	account_info_dict = mt5.account_info()._asdict()
	for prop in account_info_dict:
		print("  {}={}".format(prop, account_info_dict[prop]))
else:
	print("failed to connect at account #{}, error code: {}".format(account, mt5.last_error()))

# get number of all financial instruments in the terminal
symbols_nr = mt5.symbols_total()
print(f'total symbols {symbols_nr}')


In [ ]:
# check the presence of active orders
orders=mt5.orders_total()
if orders>0:
    print("Total orders=",orders)
else:
    print("Orders not found")
 


In [ ]:
# check the presence of open positions
positions_total=mt5.positions_total()
if positions_total>0:
    print("Total positions=",positions_total)
else:
    print("Positions not found")

In [ ]:

positions=mt5.positions_get()
if positions==None:
    print("No positions, error code={}".format(mt5.last_error()))
elif len(positions)>0:
    print("Total positions =",len(positions))
    # display all open positions
    #for position in positions:
    #    print(position)

## open positions

In [ ]:
# display these positions as a table using pandas.DataFrame
df_positions=pd.DataFrame(list(positions),columns=positions[0]._asdict().keys())
df_positions['time'] = pd.to_datetime(df_positions['time'], unit='s')
df_positions.drop(['time_update', 'time_msc', 'time_update_msc', 'external_id'], axis=1, inplace=True)
df_positions

In [ ]:
# get the number of orders in history
from_date=datetime(2020,1,1)
to_date=datetime.now()
history_orders=mt5.history_orders_total(from_date, datetime.now())
if history_orders>0:
    print("Total history orders=",history_orders)
else:
    print("Orders not found in history")

In [ ]:
position_history_orders=mt5.history_orders_get(from_date, datetime.now())
if position_history_orders==None:
    #print("No orders with position #{}".format(position_id))
    print("error code =",mt5.last_error())
elif len(position_history_orders)>0:
    #print("Total history orders on position #{}: {}".format(position_id,len(position_history_orders)))
    # display all historical orders having a specified position ticket
    #for position_order in position_history_orders:        
    #    print(position_order)
    #print()
    # display these orders as a table using pandas.DataFrame
    df=pd.DataFrame(list(position_history_orders),columns=position_history_orders[0]._asdict().keys())
    df.drop(['time_expiration','type_time','state','position_by_id','reason','volume_current','price_stoplimit','sl','tp'], axis=1, inplace=True)
    df['time_setup'] = pd.to_datetime(df['time_setup'], unit='s')
    df['time_done'] = pd.to_datetime(df['time_done'], unit='s')

## history orders

In [ ]:
df

In [ ]:
position_history_orders=mt5.history_orders_get()


In [ ]:
from_date=datetime(2020,1,1)
to_date=datetime.now()
deals=mt5.history_deals_total(from_date, to_date)
if deals>0:
    print("Total deals=",deals)
else:
    print("Deals not found in history")

In [ ]:
# get deals for symbols whose names contain neither "EUR" nor "GBP"
deals = mt5.history_deals_get(from_date, to_date)
if deals == None:
    print("No deals, error code={}".format(mt5.last_error()))
elif len(deals) > 0:
    print("history_deals_get(from_date, to_date, group=\"*,!*EUR*,!*GBP*\") =", len(deals))
    # display all obtained deals 'as is'
    #for deal in deals:
    #    print("  ",deal)
    #print()
    # display these deals as a table using pandas.DataFrame
    df=pd.DataFrame(list(deals),columns=deals[0]._asdict().keys())
    df['time'] = pd.to_datetime(df['time'], unit='s')

## history deals

In [ ]:
df

## stats

In [ ]:
df.profit.sum() # all transactions, including my inputs extraordinary payments and closed positions

In [ ]:
df.commission.sum() # fees paid on any transactions, positions open or closed

In [ ]:
df_positions.profit.sum() # current profit on open positions

In [ ]:
current_state = df.profit.sum() + df.commission.sum() + df_positions.profit.sum()
current_state # overal state if I would now close all positions

In [ ]:
my_inputs = df.loc[df['comment'] == 'Transfer from CW|2208917', 'profit'].sum()
my_inputs # my inputs

In [ ]:
profit = current_state - my_inputs
profit

In [ ]:
profit_perc = round(profit/my_inputs*100, 2)
f'{profit_perc} %'

# bitcoins

In [ ]:
btc_transactions = pd.read_csv('btc_transactions.csv')
btc_transactions

In [ ]:
# login to my account and print account info
account=z[1]['account_number']
server = z[1]['server']
password = z[1]['password']
authorized=mt5.login(login = account,server = server, password=password)
if authorized:
	# display trading account data in the form of a list
	print("Show account_info()._asdict():")
	account_info_dict = mt5.account_info()._asdict()
	for prop in account_info_dict:
		print("  {}={}".format(prop, account_info_dict[prop]))
else:
	print("failed to connect at account #{}, error code: {}".format(account, mt5.last_error()))

# get number of all financial instruments in the terminal
symbols_nr = mt5.symbols_total()
print(f'total symbols {symbols_nr}')

In [ ]:
tick = mt5.symbol_info_tick("BTCEUR")
tick

In [ ]:
s = pd.Series(tick._asdict())
s['time'] = pd.to_datetime(s['time'], unit='s')
s

## stats

In [ ]:
current_state_btc = btc_transactions['total BTC'].sum() * s['bid']
current_state_btc

In [ ]:
profit_perc_btc = round((current_state_btc - btc_transactions['amount EUR'].sum())/btc_transactions['amount EUR'].sum() * 100, 2)
f'{profit_perc_btc} %'

In [ ]:
# with open('logins.json', 'w') as fp:
    # json.dump(z, fp)